In [ ]:
import os

import pandas as pd


def get_price(data: pd.DataFrame()) -> pd.DataFrame():
    game_price_data = data.copy()
    game_price_data["Price"] = game_price_data["Price"].str.split(" - ", expand=True)[1]
    game_price_data["Price"] = game_price_data["Price"].str.replace(r'[^\d.]', '', regex=True)
    game_price_data = game_price_data.dropna(subset=["Price"])
    game_price_data["Price"] = game_price_data["Price"].astype(float)
    game_price_data["Country"] = game_price_data["Country"].str.upper()
    return game_price_data


def get_result_data(game_price_data: pd.DataFrame(), currency_data: pd.DataFrame()) -> pd.DataFrame():
    currency_data["Country"] = currency_data["Country"].str.upper().astype(str)
    result = pd.merge(game_price_data, currency_data, on="Country", how = "left")
    result = result.dropna(subset=["Visa Currency", "Master Currency", "JCB Currency"])
    result["Visa Price"] = result["Price"] * result["Visa Currency"]
    result["Master Price"] = result["Price"] * result["Master Currency"]
    result["JCB Price"] = result["Price"] * result["JCB Currency"]
    cols = [f for f in result.columns if f not in ["Country", "Currency", "Price", "Visa Currency", "Master Currency", "JCB Currency"]]
    result = result[cols]
    return result


def get_all_results(game_price_data_path: str, currency_data_path: str):
    prices = [f for f in os.listdir(game_price_data_path) if f.endswith(".csv")]
    currency = pd.read_csv(currency_data_path)
    for price in prices:
        price_path = f"{game_price_data_path}/{price}"
        data = pd.read_csv(price_path)
        game_price_data = get_price(data)
        result = get_result_data(game_price_data, currency)
        if not result.empty:
            os.makedirs("../data/games_price_result", exist_ok=True)
            result.to_csv(f"../data/games_price_result/{price}", index=False)
        else:
            pass


data = pd.read_csv("../data/games_price/約旦_info.csv")
currency = pd.read_csv("../data/currency_rate/currency_rate.csv")

game_price_data_path = "../data/games_price/"
currency_data_path = "../data/currency_rate/currency_rate.csv"
result = get_all_results(game_price_data_path, currency_data_path)
result

In [ ]:
import orjson

# 讀取原始檔案
with open("../configs/countries.json") as f:
    countries = orjson.loads(f.read())

# 創建一個集合來存儲獨特的貨幣代碼
unique_currencies = set()
# 用於存儲不重複的貨幣資訊
unique_currency_data = []

for country in countries:
    currency = country["Currency"]
    if currency not in unique_currencies:
        unique_currencies.add(currency)
        unique_currency_data.append(country)

# 現在將獨特的貨幣資訊寫入到新文件中
with open("../configs/unique_currencies.json", "w") as f:
    f.write(orjson.dumps(unique_currency_data).decode("utf-8"))

In [ ]:
import orjson


def get_country_list():
    with open("../configs/countries.json", "rb") as file:
        json_data = file.read()
        data = orjson.loads(json_data)
    return data


target_countries = get_country_list()

for i, target_country in enumerate(target_countries):
    print(target_country)